In [62]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedShuffleSplit
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
import joblib
from xgboost import XGBRegressor


In [2]:
df = pd.read_csv('clean.csv')

In [3]:
df.head()

,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,size,founded,type_of_ownership,...,Python,Bachelor,Master,No_degree,job_city,job_state,hq_city,hq_state,age,age_bin
0,Data Analyst,37 - 66,Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,...,1,0,0,1,New York,NY,New York,NY,63,old
1,Quality Data Analyst,37 - 66,Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,...,0,1,1,0,New York,NY,New York,NY,131,very old
2,Senior Data Analyst,37 - 66,We’re looking for a Senior Data Analyst who ha...,3.4,Squarespace,"New York, NY","New York, NY",1001 to 5000 employees,2003,Company - Private,...,1,1,1,0,New York,NY,New York,NY,21,new
3,Data Analyst,37 - 66,Requisition NumberRR-0001939\nRemote:Yes\nWe c...,4.1,Celerity,"New York, NY","McLean, VA",201 to 500 employees,2002,Subsidiary or Business Segment,...,0,1,0,0,New York,NY,McLean,VA,22,new
4,Reporting Data Analyst,37 - 66,ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,3.9,FanDuel,"New York, NY","New York, NY",501 to 1000 employees,2009,Company - Private,...,1,1,0,0,New York,NY,New York,NY,15,new


In [4]:
df.columns

Index(['job_title', 'salary_estimate', 'job_description', 'rating',
       'company_name', 'location', 'headquarters', 'size', 'founded',
       'type_of_ownership', 'sector', 'revenue', 'min_salary', 'max_salary',
       'average_salary', 'average_salary_bin', 'SQL', 'Excel', 'Tableau', 'R',
       'Power BI', 'SAS', 'Word', 'PowerPoint', 'AWS', 'Python', 'Bachelor',
       'Master', 'No_degree', 'job_city', 'job_state', 'hq_city', 'hq_state',
       'age', 'age_bin'],
      dtype='object')

In [5]:
df['revenue'].unique()

array(['$100 to $500 million (USD)', '$2 to $5 billion (USD)',
       'Unknown / Non-Applicable', '$50 to $100 million (USD)',
       '$1 to $2 billion (USD)', '$5 to $10 billion (USD)',
       '$1 to $5 million (USD)', '$25 to $50 million (USD)',
       '$10+ billion (USD)', 'Less than $1 million (USD)',
       '$10 to $25 million (USD)', '$500 million to $1 billion (USD)',
       '$5 to $10 million (USD)'], dtype=object)

In [6]:
columns_to_drop = ['salary_estimate','job_description','location','headquarters','founded','min_salary','max_salary']

In [7]:
categorical_features = ['job_title','company_name','type_of_ownership','sector','job_city','job_state','hq_city','hq_state']

In [8]:
df[categorical_features].head()

,job_title,company_name,type_of_ownership,sector,job_city,job_state,hq_city,hq_state
0,Data Analyst,Vera Institute of Justice,Nonprofit Organization,Non-Profit,New York,NY,New York,NY
1,Quality Data Analyst,Visiting Nurse Service of New York,Nonprofit Organization,Health Care,New York,NY,New York,NY
2,Senior Data Analyst,Squarespace,Company - Private,Information Technology,New York,NY,New York,NY
3,Data Analyst,Celerity,Subsidiary or Business Segment,Information Technology,New York,NY,McLean,VA
4,Reporting Data Analyst,FanDuel,Company - Private,"Arts, Entertainment & Recreation",New York,NY,New York,NY


In [9]:
numeric_features = ['age']

In [10]:
size_encoder = OrdinalEncoder(categories=[['Unknown','1 to 50 employees','51 to 200 employees','201 to 500 employees','501 to 1000 employees','1001 to 5000 employees','5001 to 10000 employees','10000+ employees']],handle_unknown="use_encoded_value", unknown_value= -1)
revenue_encoder = OrdinalEncoder(categories=[['Unknown / Non-Applicable','Less than $1 million (USD)','$1 to $5 million (USD)','$5 to $10 million (USD)','$10 to $25 million (USD)','$25 to $50 million (USD)','$50 to $100 million (USD)','$100 to $500 million (USD)','$500 million to $1 billion (USD)',\
                                 '$1 to $2 billion (USD)','$2 to $5 billion (USD)','$5 to $10 billion (USD)','$10+ billion (USD)'
                                 ]],handle_unknown="use_encoded_value", unknown_value= -1)

#average_salary_bin_encoder = OrdinalEncoder(categories= [['low','medium','high']],handle_unknown="use_encoded_value", unknown_value= -1)
age_encoder = OrdinalEncoder(categories= [['Unknown','new','old','very old']], handle_unknown="use_encoded_value", unknown_value= -1)

In [11]:
#y = df['average_salary']

In [12]:
num_bins = 10
df['average_salary_bin'] = pd.cut(df['average_salary'], bins=num_bins, labels=False)

In [13]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df, df['average_salary_bin']):
    strat_train_set = df.iloc[train_index]
    strat_test_set = df.iloc[test_index]

In [14]:
strat_train_set = strat_train_set.drop(columns=['average_salary_bin'])
strat_test_set = strat_test_set.drop(columns=['average_salary_bin'])

In [15]:
X_train = strat_train_set.drop(columns=['average_salary'])
y_train = strat_train_set['average_salary']
X_test = strat_test_set.drop(columns=['average_salary'])
y_test = strat_test_set['average_salary']

In [16]:
#X = df.drop(['average_salary','average_salary_bin'],axis = 1)

In [17]:
#X.drop(columns_to_drop, axis = 1, inplace = True)

In [18]:
X_train.drop(columns_to_drop, axis = 1, inplace = True)
X_test.drop(columns_to_drop, axis = 1, inplace = True)

In [19]:
#x_train, x_, y_train, y_ = train_test_split(X, y, test_size = 0.4, random_state = 42)
#x_val, x_test, y_val, y_test = train_test_split(x_,y_, test_size = 0.5, random_state = 42)
#del x_,y_

In [20]:
ct = ColumnTransformer(
    [
    ('standard_scaler', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features),
    ('size_encoder', size_encoder, ['size']),
    ('revenue_encoder', revenue_encoder, ['revenue']),
    ('age_encoder', age_encoder, ['age_bin']),
    ], remainder='passthrough'
)


In [22]:
n = OneHotEncoder(handle_unknown='ignore').fit_transform(X_train[categorical_features]).toarray()

In [23]:
#n

In [24]:
X_train.columns

Index(['job_title', 'rating', 'company_name', 'size', 'type_of_ownership',
       'sector', 'revenue', 'SQL', 'Excel', 'Tableau', 'R', 'Power BI', 'SAS',
       'Word', 'PowerPoint', 'AWS', 'Python', 'Bachelor', 'Master',
       'No_degree', 'job_city', 'job_state', 'hq_city', 'hq_state', 'age',
       'age_bin'],
      dtype='object')

In [25]:
ct.fit_transform(X_train)

<1800x2754 sparse matrix of type '<class 'numpy.float64'>'
	with 28696 stored elements in Compressed Sparse Row format>

In [26]:
feature_selection_model = RandomForestRegressor(n_jobs = -1, n_estimators= 100, random_state=42)
pipeline = Pipeline([
    ('preprocessing', ct),
    ('feature_selection', SelectFromModel(estimator=feature_selection_model, threshold='median')),
])


In [27]:
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standard_scaler',
                                                  StandardScaler(), ['age']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['job_title', 'company_name',
                                                   'type_of_ownership',
                                                   'sector', 'job_city',
                                                   'job_state', 'hq_city',
                                                   'hq_state']),
                                                 ('size_encoder',
                                                  OrdinalEncoder(categories=[['Unknown',
                                                                              '1 '
                                                                              'to '
                                                                              '50 '
                                                                              'e...
                                                                              'billion '
                                                                              '(USD)']],
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['revenue']),
                                                 ('age_encoder',
                                                  OrdinalEncoder(categories=[['Unknown',
                                                                              'new',
                                                                              'old',
                                                                              'very '
                                                                              'old']],
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['age_bin'])])),
                ('feature_selection',
                 SelectFromModel(estimator=RandomForestRegressor(n_jobs=-1,
                                                                 random_state=42),
                                 threshold='median'))])

In [28]:
feature_selection_step = pipeline.named_steps['feature_selection']
selected_feature_indices = feature_selection_step.get_support(indices=True)
len(selected_feature_indices)
#selected_feature_names = x_train.columns[selected_feature_indices].tolist()

1377

In [30]:
pipeline.fit(X_train, y_train)
print("Shape of x_train:", X_train.shape)
print("Feature selection step:", pipeline.named_steps['feature_selection'])

Shape of x_train: (1800, 26)
Feature selection step: SelectFromModel(estimator=RandomForestRegressor(n_jobs=-1, random_state=42),
                threshold='median')


In [31]:
feature_selection_step = pipeline.named_steps['feature_selection']
selected_feature_indices = feature_selection_step.get_support(indices=True)

In [32]:
numeric_features_names = ct.transformers_[0][2] 
categorical_features_names = ct.transformers_[1][1].get_feature_names_out(categorical_features) 
ordinals_features = ['size', 'revenue', 'age']
remaining_features = ct.transformers_[5][1].get_feature_names_out()

In [33]:
selected_feature_names = []
selected_feature_names.extend(numeric_features_names)
selected_feature_names.extend(categorical_features_names)
selected_feature_names.extend(ordinals_features)
selected_feature_names.extend(remaining_features)

In [34]:
len(selected_feature_names)

2754

In [35]:
print(selected_feature_indices)

[   0    1    2 ... 2751 2752 2753]


In [36]:
for idx in selected_feature_indices:
    print(selected_feature_names[idx])

age
job_title_ Summer Analyst Program
job_title_# Division Data And Financial Analyst
job_title_.Net
job_title_.Net Developer
job_title_Acoustics Data Analyst/Flight Test Engineer
job_title_Ai Insights Data Analyst
job_title_Analyst Data Intelligence (Us)
job_title_Apple Media Products (Amp)
job_title_Application Data Analyst
job_title_Application Programmer V/ Database Developer/ Data Analyst
job_title_Associate Player Support Data Analyst
job_title_Associate Salesforce.Com Data Steward & Analyst
job_title_Automotive Data Analyst
job_title_Aws Analys/ Data Analyst
job_title_Bi Developer
job_title_Bi/Data Analyst
job_title_Big Data Analyst
job_title_Bilingual (Chinese) Data Analyst
job_title_Biological Data Analyst
job_title_Biomedical Data Analyst I
job_title_Business Analyst
job_title_Cbo Data Analyst
job_title_Ccpa Data Analyst
job_title_Certified Tumor Registrar/Data Analyst [Ctr And Experience Required]
job_title_Ci Data Analyst/Engineer
job_title_Client Data Analyst
job_title_Cli

In [37]:
best_features = ['age','job_title','company_name','type_of_ownership','sector','job_city','job_state','hq_city','hq_state','size','revenue','age','rating',\
                'SQL','Excel','Tableau','R','Power BI','SAS','Word','PowerPoint','AWS','Python','Bachelor','Master','No_degree'
                ]

In [38]:
len(selected_feature_names)

2754

In [39]:
len(selected_feature_indices)

1377

In [40]:
len(best_features)

26

In [55]:
param_grids = {
    'SVR': {
        'model__C': [0.1, 1, 10],
        'model__epsilon': [0.01, 0.1, 0.5],
        'model__kernel': ['linear', 'poly', 'rbf']
    },
    'Decision Tree': {
        'model__max_depth': [None, 5, 10, 20],
        'model__min_samples_split': [2, 5, 10]
    },
    'Random Forest': {
        'model__n_estimators': [100, 200],
        'model__max_depth': [None, 10, 20],
        'model__min_samples_split': [2, 5]
    },
    'ElasticNet': {
        'model__alpha': [0.1, 0.5, 1.0],
        'model__l1_ratio': [0.1, 0.5, 0.9]
    },
    'KNN': {
        'model__n_neighbors': [3, 5, 7],
        'model__weights': ['uniform', 'distance']
    },
    'Gradient Boosting': {
        'model__n_estimators': [100, 200],
        'model__learning_rate': [0.01, 0.1, 0.2],
        'model__max_depth': [3, 5, 7]
    }
}

models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(max_iter=5000),
    'ElasticNet': ElasticNet(),
    'SVR': SVR(),
    'KNN Regression': KNeighborsRegressor(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor()
}


In [56]:
best_model = None
best_score = float('-inf')

def evaluate_model_with_grid_search(model, model_name, param_grid=None):
    global best_model, best_score
    pipeline = Pipeline(steps=[
        ('preprocessor', ct),
        ('feature_selection', SelectFromModel(estimator=feature_selection_model, threshold='median')),
        ('model', model)
    ])
    
    if param_grid:
        grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=5, n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_estimator = grid_search.best_estimator_
        r_train = best_estimator.score(X_train, y_train)
        r_test = best_estimator.score(X_test, y_test)
        y_train_pred = best_estimator.predict(X_train)
        y_test_pred = best_estimator.predict(X_test)
        mae_train = mean_absolute_error(y_train, y_train_pred)
        mae_test = mean_absolute_error(y_test, y_test_pred)
        print(f'{model_name}:')
        print(f'  Best Params: {grid_search.best_params_}')
        print(f'  Train r-score = {r_train:.4f}')
        print(f'  test r-score = {r_test:.4f}')
        print(f'  Train MAE = {mae_train:.4f}')
        print(f'  Test MAE = {mae_test:.4f}')
        if r_test > best_score:
            best_score = r_test
            best_model = best_estimator
    else:
        pipeline.fit(X_train, y_train)
        r_train = pipeline.score(X_train, y_train)
        r_test = pipeline.score(X_test, y_test)
        y_train_pred = pipeline.predict(X_train)
        y_test_pred = pipeline.predict(X_test)
        mae_train = mean_absolute_error(y_train, y_train_pred)
        mae_test = mean_absolute_error(y_test, y_test_pred)
        print(f'{model_name}:')
        print(f'  Train r-score = {r_train:.4f}')
        print(f'  val r-score = {r_test:.4f}')
        print(f'  Train MAE = {mae_train:.4f}')
        print(f'  Test MAE = {mae_test:.4f}')
        if r_test > best_score:
            best_score = r_test
            best_model = pipeline


In [57]:
for model_name, model in models.items():
    param_grid = param_grids.get(model_name, None)
    evaluate_model_with_grid_search(model, model_name, param_grid)

Linear Regression:
  Train r-score = 0.8731
  val r-score = 0.0729
  Train MAE = 4.7468
  Test MAE = 16.4348
Ridge Regression:
  Train r-score = 0.7828
  val r-score = 0.3100
  Train MAE = 8.3773
  Test MAE = 14.8864
Lasso Regression:
  Train r-score = 0.1853
  val r-score = 0.1985
  Train MAE = 16.1906
  Test MAE = 16.0036
ElasticNet:
  Best Params: {'model__alpha': 0.1, 'model__l1_ratio': 0.9}
  Train r-score = 0.3319
  test r-score = 0.3294
  Train MAE = 14.7180
  Test MAE = 14.7695
SVR:
  Best Params: {'model__C': 1, 'model__epsilon': 0.5, 'model__kernel': 'linear'}
  Train r-score = 0.4024
  test r-score = 0.3149
  Train MAE = 12.6273
  Test MAE = 13.9913
KNN Regression:
  Train r-score = 0.3989
  val r-score = 0.0843
  Train MAE = 13.8572
  Test MAE = 17.1532
Decision Tree:
  Best Params: {'model__max_depth': 5, 'model__min_samples_split': 10}
  Train r-score = 0.3814
  test r-score = 0.3190
  Train MAE = 14.0801
  Test MAE = 14.8239
Random Forest:
  Best Params: {'model__max_dep

In [59]:
pipeline = Pipeline(steps=[
        ('preprocessor', ct),
        ('feature_selection', SelectFromModel(estimator=feature_selection_model, threshold='median')),
        ('model', XGBRegressor())
    ])

param_grid = {
    'model__learning_rate': [0.1, 0.3],
    'model__max_depth': [3, 5, 7],
    'model__n_estimators': [50, 100, 200]
}

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_error', verbose=1)

grid_search.fit(X_train, y_train)

# Print best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best CV Score:", -grid_search.best_score_)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
Best Parameters: {'model__learning_rate': 0.3, 'model__max_depth': 3, 'model__n_estimators': 50}
Best CV Score: 14.655068895551892


In [60]:
best_pipeline = grid_search.best_estimator_
best_pipeline.fit(X_train, y_train)
y_pred = best_pipeline.predict(X_test)

In [63]:
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Test MAE: {mae:.4f}')
print(f'Test R-squared: {r2:.4f}')

Test MAE: 13.9745
Test R-squared: 0.3965


In [65]:
if best_model:
    joblib.dump(best_pipeline, 'xgboost_best_pipeline.pkl')
    print('Best model saved to xg_boost.pkl')

Best model saved to xg_boost.pkl
